In [1]:
import numpy as np
import geopandas as gpd
import rasterio
import folium
from folium.plugins import HeatMap
from folium.plugins.treelayercontrol import TreeLayerControl
from folium.raster_layers import WmsTileLayer, ImageOverlay
from tqdm import tqdm # impatient
# was using rasterio to create raster heatmap, may still be used but not sure
from rasterio.features import rasterize
from rasterio.plot import reshape_as_image
from rasterio.warp import calculate_default_transform, reproject
import matplotlib.pyplot as plt
from PIL import Image
import tempfile

In [2]:
GEOSERVER_URL = "https://sparcal.sdsc.edu/geoserver/rrk_sn/wms?"
# directory information for local data
DATA_DIR = "data/"
UTILS_DIR = "utils/"
SN_DIR = DATA_DIR + "sierra_nevada/"
# pathnames
IGNITION_PATH = SN_DIR + "sierra_nevada_fire_ignitions.geojson"
PERIMETER_PATH = SN_DIR + "sierra_nevada_fire_perimeters.geojson"
UTILITY_PATH = SN_DIR + "sierra_nevada_utility_lines.geojson"
SN_SHAPE_PATH = UTILS_DIR + "sierra_nevada_boundary.geojson"
SN_SHAPE = gpd.read_file(SN_SHAPE_PATH).to_crs(epsg=4326)['geometry']

# Fire/Utility Data


In [3]:
# read in utility data
utility_df = gpd.read_file(UTILITY_PATH).to_crs(epsg=4326)
# read in fire perimeter data
utility_perimeter_df = gpd.read_file(PERIMETER_PATH).to_crs(epsg=4326) # fire perimeters intersecting with utility lines
all_perimeter_df = gpd.read_file("./data/all_sn_fire_perimeters.geojson") # all fire perimeters
for column in all_perimeter_df.select_dtypes(include=['datetime', 'datetime64']).columns:
    all_perimeter_df[column] = all_perimeter_df[column].astype(str)
# read in ignition data
ignition_df = gpd.read_file(IGNITION_PATH).to_crs(epsg=4326)    
buffered_ignition_df = gpd.read_file(IGNITION_PATH) # TODO: optimize for memory (redundant data)
buffered_ignition_df['geometry'] = buffered_ignition_df.buffer(500)
buffered_ignition = buffered_ignition_df.to_crs(epsg=4326)

/opt/anaconda3/envs/wired-utility/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/anaconda3/envs/wired-utility/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [4]:
# folium layer for utility lines
utility_layer = folium.GeoJson(
    utility_df, 
    name="Transmission Lines",
    control=False
)
# Fire Perimeters
def perimeter_style(feature): # style
    color = 'red'
    opacity = 0.2
    return {
        'fillColor': color,
        'color': color,
        'weight': 4,
        'fillOpacity': opacity,
        'opacity': opacity,
    }
perimeter_layer = folium.GeoJson(
    all_perimeter_df,
    name="Fire Perimeters",
    style_function=perimeter_style
)
# Fire Ignition Points
def ignition_style(feature): # style
    color = 'brown'
    opacity = 0.3
    return {
        'fillColor': color,
        'color': color,
        'weight': 4,
        'fillOpacity': opacity,
        'opacity': opacity,
    }
ignition_layer = folium.GeoJson(
    buffered_ignition_df,
    name="Fire Ignition Points",
    style_function=ignition_style,
    #marker=folium.CircleMarker(),
)
# Folium Heatmap for Ignition Points (unused but here's the code)
""" ignition_df['LATITUDE'] = ignition_df.geometry.y
ignition_df['LONGITUDE'] = ignition_df.geometry.x
ignition_coords = ignition_df[['LATITUDE', 'LONGITUDE']].values.tolist()
ignition_heatmap_layer = HeatMap(
    ignition_coords,
    name="Ignition Points",
    max_zoom=10,
    ) """

# Heatmaps with Vector To Heatmap Overlay Function
"""
perimeter_heatmap = vector_to_heatmap_overlay(perimeter_df['geometry'], "Fire Perimeters")
ignition_heatmap = vector_to_heatmap_overlay(buffered_ignition, "Fire Ignition Points")
sn_raster = vector_to_heatmap_overlay([SN_SHAPE], "Sierra Nevada Raster", cmap_name='viridis')
"""

# save all to list
fire_layers = [
    {'label': 'Fire Perimeters', 'layer': perimeter_layer},
    {'label': 'Fire Ignition Points','layer': ignition_layer}
]

# Geoserver Layers
RRK link: https://rrk.sdsc.edu/sierra.html

Geoserver Link: https://sparcal.sdsc.edu/geoserver/web/wicket/bookmarkable/org.geoserver.web.demo.MapPreviewPage?0&filter=false

In [16]:
carbon_layers = []
carbon_layer_info = {
    'Dead Carbon':'rrk_sn:DeadCarbon_2021',
    'Total Carbon':'rrk_sn:CECS_TotalCarbon_30m',
    'Aboveground Live Tree Carbon':'rrk_sn:F3_AbovegroundLiveTreeCarbon_2021',
    'Total Aboveground Carbon':'rrk_sn:CECS_TotalCarbon_30m'
}
for layer_name in carbon_layer_info:
    layer = WmsTileLayer(
        url=GEOSERVER_URL,
        layers=carbon_layer_info[layer_name],
        name=layer_name,
        transparent=True,
        fmt="image/png8",
        overlay=True,
        control=True,
        show=False,
        tiled=True,
        opacity=0.5,
    )
    layer_dict = {'label': layer_name, 'layer': layer}
    carbon_layers.append(layer_dict)

# The Map

In [18]:
min_lon, min_lat, max_lon, max_lat = SN_SHAPE[0].bounds
center = SN_SHAPE[0].centroid
map_center = [center.y, center.x]

m = folium.Map(
    location=map_center, 
    tiles="CartoDB Positron", 
    zoom_start=8,
    control_scale=True,
    min_lat=min_lat, # does this do anything?
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
)

""" #kind of an eyesore
folium.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner", color="green").add_to(m)
folium.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(m)
folium.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(m)
folium.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(m) """

utility_layer.add_to(m)
layers = fire_layers + carbon_layers
for layer in layers:
    layer['layer'].add_to(m)

overlay_tree = [
    {'label': 'Fire Layers', 'children': fire_layers},
    {'label': 'Carbon Layers', 'children': carbon_layers}
]

#folium.GeoJson(SN_SHAPE, name="Sierra Nevada Region").add_to(m)
TreeLayerControl(overlay_tree=overlay_tree).add_to(m)
m.save('map.html')
display(m)